In [10]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from utils import *

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd

from datetime import datetime
from collections import OrderedDict

In [2]:
PATH = Path("/data2/yinterian/multi-task-romain")

In [3]:
train, valid = get_data_gap(PATH, gap="15min")

In [4]:
train.shape, valid.shape

((36219, 17), (4000, 17))

In [13]:
norm_dict, care2id, id2index, num_subjects = stats_dict(train)

In [14]:
train_ds = MultiTask(train, norm_dict, id2index, care2id)
valid_ds = MultiTask(valid, norm_dict, id2index, care2id, train=False)

In [15]:
batch_size = 5000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [20]:
def multiple_runs(num_subjects, single, basename="model_15min", n=10):
    best_score = 0
    for i in range(n):
        model = EventModel(num_subjects, single=single).cuda()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.04, weight_decay=1e-5)
        score = train_epochs(model, train_ds, valid_dl, optimizer, filename=basename, epochs=15, C=10)
        if score > best_score:
            best_score = score
        update_optimizer(optimizer, lr=0.02)
        score = train_epochs(model, train_ds, valid_dl, optimizer, filename=basename, epochs=25, C=10)
        if score > best_score:
            best_score = score
    return best_score    

In [ ]:
multiple_runs(num_subjects, single=False, basename="model_15min", n=10)

	Train loss: 7117.427 6043.509 map valid loss: 9653.976 valid r2 hr -42.736 valid r2 map -60.497
	Train loss: 6290.207 4994.721 map valid loss: 3573.435 valid r2 hr -20.723 valid r2 map -21.763
	Train loss: 4292.257 3074.083 map valid loss: 1190.125 valid r2 hr -8.548 valid r2 map -6.581
	Train loss: 1754.260 904.642 map valid loss: 72.616 valid r2 hr -1.390 valid r2 map 0.537
	Train loss: 369.494 81.292 map valid loss: 322.290 valid r2 hr -0.563 valid r2 map -1.053
	Train loss: 423.439 331.805 map valid loss: 213.431 valid r2 hr -0.751 valid r2 map -0.360
	Train loss: 315.726 94.351 map valid loss: 32.276 valid r2 hr 0.033 valid r2 map 0.794
/data2/yinterian/multi-task-romain/models/model_15min_r2_3_79.pth
	Train loss: 238.805 45.051 map valid loss: 80.302 valid r2 hr 0.004 valid r2 map 0.488
	Train loss: 193.991 61.471 map valid loss: 34.998 valid r2 hr 0.540 valid r2 map 0.777
	Train loss: 110.897 25.795 map valid loss: 30.680 valid r2 hr 0.668 valid r2 map 0.805
/data2/yinterian/mu

## Single model

In [ ]:
def multiple_runs_single(num_subjects, single, basename="model_15min", n=10):
    best_score = 0
    for i in range(n):
        model = EventModel(num_subjects, single=single).cuda()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.04, weight_decay=1e-5)
        score = train_epochs_map(model, train_ds, valid_dl, optimizer, filename=basename, epochs=15)
        if score > best_score:
            best_score = score
        update_optimizer(optimizer, lr=0.02)
        score = train_epochs_map(model, train_ds, valid_dl, optimizer, filename=basename, epochs=25)
        if score > best_score:
            best_score = score
    return best_score 

In [ ]:
multiple_runs_single(num_subjects, True, basename="model_single_15min", n=10)